In [1]:
import time
from termcolor import colored
import torch

from modules.Dcls2d import Dcls2d
from modules.Dcls2d_old import Dcls2d_old

assert torch.cuda.is_available()
cuda_device = torch.device("cuda")  # device object representing GPU

in_channels = 1
out_channels = 1
kernel_size = (2,2)
dilation = (2,2)
stride = (1,1)
padding = (0,0)
groups = 1
bias = False

m = torch.nn.Conv2d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)

n = Dcls2d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)

X1 = torch.nn.Parameter(
                      torch.tensor([[[[1., 2., 3., 4.],
                                    [5., 6., 7., 8.], 
                                    [9., 10., 11., 12.],
                                    [13., 14., 15., 16.]]]],device=cuda_device),
                      requires_grad = True) 
X2 = torch.nn.Parameter(
                      torch.tensor([[[[1., 2., 3., 4.],
                                    [5., 6., 7., 8.], 
                                    [9., 10., 11., 12.],
                                    [13., 14., 15., 16.]]]],device=cuda_device),
                      requires_grad = True) 
m.weight = torch.nn.Parameter(
                      torch.tensor([[[[20., 40.],
                                    [60., 80.]]]],device=cuda_device),
                      requires_grad = True)
n.weight = torch.nn.Parameter(
                      torch.tensor([[[[20., 40.],
                                    [60., 80.]]]],device=cuda_device),
                      requires_grad = True)

height_out = (4 + 2 * padding[0] - (dilation[0] * (kernel_size[0] - 1) + 1)) / stride[0] + 1;
width_out = (4 + 2 * padding[1] - (dilation[1] * (kernel_size[1] - 1) + 1)) / stride[1] + 1;
back_truth = torch.nn.Parameter(
                      torch.tensor([[[[1., 2.],
                                    [4., 5.]]]],device=cuda_device),
                      requires_grad = True)

var1 = (m(X1) - back_truth).norm()
var2 = (n(X2) - back_truth).norm()
#var3 = (o(Y) - backtruth).norm()


var1.backward();
print(m.weight.grad)
var2.backward();

tensor([[[[ 7.6718, 11.5944],
          [23.3621, 27.2847]]]], device='cuda:0')


In [2]:
print(X1.size())
print(m.weight.size())
print(n.weight.size())

print(m(X1).size())
print(m(X1))
print(n(X2).size())
print(n(X2))

R1 = torch.ceil(n.P1) - n.P1
R2 = torch.ceil(n.P2) - n.P2

print((1 - R1) * (1-R2))
print((R1) * (1-R2))
print((1 - R1) * (R2))
print((R1) * (R2))
'''print(m.weight.grad) 
print(n.weight.grad)'''

torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 2, 2])
tensor([[[[1560., 1760.],
          [2360., 2560.]]]], device='cuda:0',
       grad_fn=<CudnnConvolutionBackward>)
torch.Size([1, 1, 2, 2])
tensor([[[[1559.9686, 1759.9686],
          [2359.9688, 2559.9688]]]], device='cuda:0',
       grad_fn=<SurrogateDilationBackward>)
tensor([[[1., 1.],
         [1., 1.]]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([[[0., 0.],
         [0., 0.]]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([[[0., 0.],
         [0., 0.]]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([[[0., 0.],
         [0., 0.]]], device='cuda:0', grad_fn=<MulBackward0>)


'print(m.weight.grad) \nprint(n.weight.grad)'

In [3]:
print(n.P1)
print(torch.ceil(n.P1) + 2)

Parameter containing:
tensor([[[-2., -2.],
         [ 0.,  0.]]], device='cuda:0', requires_grad=True)
tensor([[[0., 0.],
         [2., 2.]]], device='cuda:0', grad_fn=<AddBackward0>)


In [4]:
def hevi(x):
    return 1.0 if x > 0 else 0.0
def surr_ceil(x,bot,top):
    s = 0.0
    
    for i in range(1-bot,top):
        s = s + hevi(x+i)
    return s - top + 1


def surr_floor(x,bot,top):
    s = 0.0
    
    for i in range(-bot,top-1):
        s = s + hevi(x+i)
    return s - bot +1

print(surr_ceil(3,5,5))
print(surr_floor(3.1,5,5))

3.0
3.0


In [5]:
import numpy as np
(np.array([12,24,36,48,3,6,9,12,4,8,12,16,1,2,3,4]) * np.array([1,3,9,11,5,7,13,15,2,4,10,12,6,8,14,16])).sum()

1770